Every notebook starts with the following three lines; they ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

We import all the necessary packages. We are going to work with the fastai V1 library which sits on top of Pytorch 1.0. The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [ ]:
!pip install fastai
from fastai.vision import *
from fastai.metrics import error_rate

# Look at data 

We are going to use the Food 101 dataset (https://www.kaggle.com/kmader/food41#1028787.jpg ) which features 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. For the purpose of this notebook, we will be classifying amongst 5 different food categories (applie pie, waffles, padthai, bread pudding and ramen) and only using 200 umages per food category.

In [ ]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
#print(ws.get_details())


In [ ]:
from azureml.core import Dataset
#Get and download food images from the File Datasets from the workspace
#Create directory and choose an appropriate folder name for your labeled images

apple_pie_dataset = Dataset.get_by_name(ws, name='apple-pie')
apple_pie_dataset.download(target_path='./data/apple-pie', overwrite=False)

waffles_dataset = Dataset.get_by_name(ws, name='waffles')
waffles_dataset.download(target_path='./data/waffles', overwrite=False)

padthai_dataset = Dataset.get_by_name(ws, name='pad-thai')
padthai_dataset.download(target_path='./data/padthai', overwrite=False)

breadpudding_dataset = Dataset.get_by_name(ws, name='bread-pudding')
breadpudding_dataset.download(target_path='./data/breadpudding', overwrite=False)

ramen_dataset = Dataset.get_by_name(ws, name='ramen')
ramen_dataset.download(target_path='./data/ramen', overwrite=False)

Choose an appropriate name for your labeled images. You can run these steps multiple times to create different labels.

In [ ]:
import pathlib
from pathlib import Path
#directories to images 
path = Path('data')
path.ls()

In [ ]:
#class for each category to classify
classes = ['apple-pie','breadpudding','padthai', 'ramen', 'waffles']

In [ ]:
#remove any images that cannot be opened 
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=2200)

# View Data

ImageDataBunch returns a data bunch object which is needed for modeling in fast AI. This contains the images and labels for the training, validation and/or test datasets. 

We want to make all images the same shape and size for better GPU performance(GPUs have to provide the same instruction to a bunch of things at the same time in order to be fast). We will use size = 224 to ensure that all the images are of 224x224 square size.

We use NumPy random seed when we need to generate pseudo-random numbers in a repeatable way*.

If we the data is not normalized, we can have difficulty training a model. We want the red, green and blue channels of the images to have a mean of 0 and standard deviation of 1. 

The get_transforms function makes the size of the image 224 x 224 by doing a combination of things like center cropping and resizing. 

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224).normalize(imagenet_stats)

In [ ]:
#let's take a look at our delicious food! Yummm....
data.show_batch(rows=5, figsize=(7,6))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

# Training 

Convolutional Neural Networks (http://cs231n.github.io/convolutional-networks/) are very similar to ordinary Neural Networks but the assumption is that the inputs are images- they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. Activation functions are applied to each node to capture non linearities in the data.

We will use a convolutional neural network backbone and a fully connected head with a single hidden layer as a classifier. We are building a model which will take images as input and will output the predicted probability for each of the categories (in this case, it will have 5 outputs).

In [ ]:
#model name is learn
learn = cnn_learner(data, models.resnet34, metrics=[error_rate])

In [ ]:
my_model = learn.fit_one_cycle(1)

In [ ]:
from azureml.core import Experiment, Run

#create new aml experiment 
experiment = Experiment(ws, 'fastai-food-classification')

In [ ]:
%%writefile train.py

from fastai.vision import (ImageDataBunch, get_transforms, cnn_learner, models, imagenet_stats, accuracy)
#from fastai.metrics import error_rate
from pathlib import Path 
from azureml.core.run import Run 
import numpy as np

# get the Azure ML run object
run = Run.get_context()

# get images
path = Path('data')
np.random.seed(2)
data = ImageDataBunch.from_folder(path,
                                       train=".",
                                       valid_pct=0.2,
                                       ds_tfms=get_transforms(),
                                       size=224).normalize(imagenet_stats)
# build estimator based on ResNet 34
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(2)
# do test time augmentation and get accuracy
acc = accuracy(*learn.TTA())
# log the accuracy to run`
run.log('Accuracy', np.float(acc))
print("Accuracy: ", np.float(acc))
# save model
learn.path = Path("./outputs")
learn.export()


In [ ]:
from azureml.core.environment import CondaDependencies
from azureml.core import Environment
from azureml.core import ScriptRunConfig

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()


conda_dep.add_pip_package("fastai")
conda_dep.add_pip_package("ipykernel")
myenv.python.conda_dependencies=conda_dep

azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 50000000000
#To submit a run, create a run configuration that combines the script file and environment, and pass it to Experiment.submit. 
#In this example, the script is submitted to local computer, but you can specify other compute targets such as remote clusters as well.


#instead of managing the setup of the environment yourself, you can ask the system to build a new conda environment for you. 
#The environment is built once, and will be reused in subsequent executions as long as the conda dependencies remain unchanged.

myenv.python.user_managed_dependencies = False
runconfig = ScriptRunConfig(source_directory=".", script="train.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = myenv
run = experiment.submit(config=runconfig)

run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails 
RunDetails(run).show()

In [ ]:
# save model
#create outputs folder in github repo
learn.path = Path("./outputs")
learn.export()
path = learn.path
path

In [ ]:
# Define remote compute target to use
# Further docs on Remote Compute Target: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-remote

# Choose a name for your cluster.
amlcompute_cluster_name = "gpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)
    
# For a more detailed view of current AmlCompute status, use get_status().

In [ ]:
#if you would like to run remotely, you can run this cell and the following two cells 

#Next, we construct an `azureml.train.dnn.PyTorch` estimator object
#The PyTorch estimator provides a simple way of launching a PyTorch training job on a compute target. 
#We'll submit the train.py file using the `entry_script` parameter and use the gpu-cluster we created above as our `compute target`
#Since fastai v1 is built on top of PyTorch 1.0, we can use the `framework_version` parameter to specify our PyTorch version, and the estimator will automatically retrieve a Docker image that has PyTorch 1.0 and its dependencies installed.
#Finally, on top of our PyTorch base image, we'll add the fastai v1 and pillow packages using the `pip_packages` parameter. The pillow package is necessary for image processing, and we'll pin to versions below 7.0.0 since those are known to be compatible with PyTorch 1.0's torchvision package
#pillow's latest version is not compatible with the latest version of fastAI
#framework version - we want to install pytorch version 1.0 only works with fastai version 1 
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=".",
                            compute_target="gpu-cluster",
                            entry_script='train.py',
                            framework_version='1.0',
                            pip_packages=['fastai', 'pillow<7'])

In [ ]:
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 500000000000
run = experiment.submit(estimator)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
# Save the model to the outputs directory for capture. Note: this is not registering model
from azureml.core import Experiment
import joblib
model_file_name = 'outputs/export1.pkl'

joblib.dump(value = my_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)


In [ ]:
#Register the model
from azureml.core.model import Model

# register the model1.pkl file provided in the output folder 
model = Model.register(model_path = "outputs/export1.pkl", # this points to a local file
                       model_name = "food_classification_model", # this is the name the model is registered as
                       tags = {'food': "Yummmm :)"},
                       description = "Model predicting types of food",
                       workspace = ws)

print(model.name, model.description, model.version)


Prepare to deploy


To deploy the model, you need the following items:

1) An entry script, this script accepts requests, scores the requests by using the model, and returns the results.
2) Dependencies, like helper scripts or Python/Conda packages required to run the entry script or model.
3) The deployment configuration for the compute target that hosts the deployed model. This configuration describes things like memory and CPU requirements needed to run the model.

1. Define your entry script and dependencies
Entry script
We will first write the entry script as shown below. Note a few points in the entry script.

The script contains two functions that load and run the model:

init(): Typically, this function loads the model into a global object. This function is run only once, when the Docker container for your web service is started.

When you register a model, you provide a model name that's used for managing the model in the registry. You use this name with the Model.get_model_path() method to retrieve the path of the model file or files on the local file system. If you register a folder or a collection of files, this API returns the path of the directory that contains those files.

run(input_data): This function uses the model to predict a value based on the input data. Inputs and outputs of the run typically use JSON for serialization and deserialization. You can also work with raw binary data. You can transform the data before sending it to the model or before returning it to the client.

In [ ]:
%%writefile score.py
import os
import json
from azureml.core.model import Model
from azureml.core import Workspace
import fastai 
from fastai.vision import *
from fastai.metrics import accuracy 

global food_classification_model

def init():   
    model_filename="export1.pkl"
    # The AZUREML_MODEL_DIR environment variable indicates a directory containing the model file you registered.  
    model_path=os.getenv('AZUREML_MODEL_DIR')
    food_classification_model = load_learner(path=model_path, file=model_filename)
  
    
    print(food_classification_model.data.classes)


def run(request):
    img = open_image(request)
    prediction = food_classification_model.predict(img)
    pred = str(prediction[0])
    print(pred)
    return pred



#if __name__ == "__main__":
    #init()
    #request = "data/apple-pie/3068872.jpg"
    #run(request)

It returns a tuple of three things: the object predicted (with the class in this instance), the underlying data (here the corresponding index) and the raw probabilities. You can also do inference on a larger set of data by adding a test set. This is done by passing an ItemList to load_learner.

learn = load_learner(mnist, test=ImageList.from_folder(mnist/'test'))
https://docs.fast.ai/tutorial.inference.html

Define dependencies
The following YAML is the Conda dependencies file we will use for inference. If you want to use automatic schema generation, your entry script must import the inference-schema packages.

In [ ]:
%%writefile myenv.yml

name: project_environment
dependencies:
- python=3.6.9

- pip:
  - fastai
  - torch
  - torchvision  
  - azureml-defaults
  - azureml-core
  - matplotlib==3.1.2
- numpy==1.16.2
- Pillow==6.2.0
- pandas==0.23.4



In [ ]:
from azureml.core import Environment

# Instantiate environment
myenv = Environment.from_conda_specification(name = "myenv",
                                             file_path = "myenv.yml")

Define your inference configuration. The inference configuration describes how to configure the model to make predictions. This configuration isn't part of your entry script. It references your entry script and is used to locate all the resources required by the deployment. It's used later, when you deploy the model.


In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

Define your deployment configuration Before deploying your model, you must define the deployment configuration. The deployment configuration is specific to the compute target that will host the web service. The deployment configuration isn't part of your entry script. It's used to define the characteristics of the compute target that will host the model and entry script.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags = {'task': "image-classification"}, 
                                               description='A model to predict types of food')

Deployment uses the inference configuration deployment configuration to deploy the models. The deployment process is similar regardless of the compute target

In [ ]:
service = Model.deploy(ws, name='image-classification-fastai14', models=[model], inference_config= inference_config, deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
service.get_logs()

In [ ]:
request = "data/apple-pie/3068872.jpg"
prediction = service.run(request)
print(prediction)


In [ ]:
import json
from PIL import Image 

import matplotlib.pyplot as plt


%matplotlib inline
plt.imshow(Image.open('data/apple-pie/3068872.jpg'))

Finally, let's test our deployed web service. We will send the data as a JSON string to the web service hosted in ACI and use the SDK's run API to invoke the service. 

In [ ]:
import torch
from torchvision import transforms
    
def preprocess(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image.numpy()

In [ ]:
import json 
import numpy as np 


input_data = preprocess('data/apple-pie/3068872.jpg')

lists = input_data.tolist()

#convert input data into JSON string 
json_str = json.dumps(lists)
print(type(json_str))
# deserializes sample to a python object 
sample = json.loads(json_str)
print(type(sample))
# serializes sample to JSON formatted string as expected by the scoring script
sample = json.dumps({"input_data":sample})
print(type(sample))
#prediction = service.run(sample)
#print(prediction)


In [ ]:
sample = "data/apple-pie/3068872.jpg"
sample = json.dumps({"request":sample})
print(sample)
print(type(sample))

prediction = service.run(sample)
print(prediction)

In [ ]:

import json

global food_classification_model

def init():
    path = learn.path
    path=os.getenv(('AZUREML_MODEL_DIR'))
    filename="export1.pkl"
    food_classification_model = load_learner(path=path, file=filename)
    print(food_classification_model.data.classes)


def run(request):
    img = open_image(request)
    prediction = food_classification_model.predict(img)
    pred = str(prediction[0])
    print(pred)
    return pred



#if __name__ == "__main__":
    #init()
    #request = "data/apple-pie/3068872.jpg"
    #run(request)
    


In [ ]:
request = "data/apple-pie/3068872.jpg"
run(request)

In [ ]:
request = "data/apple-pie/3068872.jpg"
prediction = service.run(request)
print(prediction)

In [ ]:
path = learn.path
print(path)

In [ ]:
  
    # note here "food_classification_model" is the name of the model registered under the workspace, this call should return the path to the model.pkl file on the local disk
    path = os.getenv('myenv')
    filename="export1.pkl"
    food_classification_model = load_learner(path=path, file=filename)
    
    print(food_classification_model.classes)    
    
    
def run(request):
    img = open_image(request)
    prediction = food_classification_model.predict(img)
    print(prediction)
    return str(prediction[0])

sample = "data/apple-pie/3068872.jpg"
sample = json.dumps({"request":sample})
print(sample)
print(type(sample))

#prediction = service.run(sample)
#print(prediction)
    

In [ ]:
#scoring using fast AI 
img = open_image(path/'apple-pie'/'3068872.jpg')
img
pred_class,pred_idx,outputs = learn.predict(img)
pred_class